In [1]:
!git clone https://github.com/Coeze/MX4553_Modelling_Theory_Project.git

Cloning into 'MX4553_Modelling_Theory_Project'...
remote: Enumerating objects: 497, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 497 (delta 50), reused 13 (delta 13), pack-reused 428 (from 4)
Receiving objects: 100% (497/497), 11.27 MiB | 10.59 MiB/s, done.
Resolving deltas: 100% (267/267), done.


In [2]:
%cd MX4553_Modelling_Theory_Project

/content/MX4553_Modelling_Theory_Project


# Model Calibration using Genetic Algorithm

This notebook implements a genetic algorithm to optimize the parameters of the forest fire spread model using real fire data from the MTBS dataset. The optimization is based on the Sørensen index (also known as the Dice coefficient), which measures the spatial agreement between simulated and observed burned areas.

In [3]:
!pip install fiona rasterio numpy pandas scikit-learn matplotlib matplotlib-scalebar geopandas pyproj shapely deap noise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 7.4 MB/s eta 0:00:00
  Created wheel for noise: filename=noise-1.2.2-cp311-cp311-linux_x86_64.whl size=56283 sha256=4f141388c34ad12106b6fd2fb6e084f2336fd8b6cd1c74c445340bb574e26dc4
  Stored in directory: /root/.cache/pip/wheels/39/25/2e/af6d1bcc91a8f99af0f651f8718b9ab999720a21c6d4149091
Successfully built noise


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from src.model import CA

from noise import snoise2

In [2]:
grid_size = (250, 250)

In [3]:
def generate_terrain_simplex(rows, cols, cell_size=30.0, min_elevation=100, max_elevation=1000,
                           octaves=6, persistence=0.5, lacunarity=2.0, scale=100.0, seed=None):
    """
    Generate realistic terrain elevation, slope, and aspect using Simplex noise.

    Parameters:
    - rows, cols: Dimensions of the grid
    - cell_size: Size of each cell in meters
    - min_elevation, max_elevation: Range of elevation values in meters
    - octaves: Number of noise layers to combine (more = more detail)
    - persistence: How much each octave contributes (amplitude multiplier)
    - lacunarity: How frequency increases with each octave
    - scale: Base scale of the noise (higher = more gradual changes)
    - seed: Random seed for reproducibility

    Returns:
    - elevation: 2D numpy array of elevation values
    - slope: 2D numpy array of slope values in degrees
    - aspect: 2D numpy array of aspect values in degrees (0-360, 0=North)
    """

    if seed is not None:
        np.random.seed(seed)

    # Generate elevation using simplex noise
    elevation = np.zeros((rows, cols))

    # For better performance, vectorize the coordinates
    y_coords = np.linspace(0, scale, rows)
    x_coords = np.linspace(0, scale, cols)

    for octave in range(octaves):
        frequency = lacunarity ** octave
        amplitude = persistence ** octave

        for i, y in enumerate(y_coords):
            for j, x in enumerate(x_coords):
                elevation[i, j] += amplitude * snoise2(
                    y * frequency / scale,
                    x * frequency / scale
                )

    # Normalize to 0-1 range
    elevation_min = elevation.min()
    elevation_max = elevation.max()
    elevation = (elevation - elevation_min) / (elevation_max - elevation_min)

    # Scale to desired elevation range
    elevation = min_elevation + elevation * (max_elevation - min_elevation)

    # Calculate slope and aspect from elevation using gradients
    dy, dx = np.gradient(elevation, cell_size, cell_size)

    # Calculate slope in degrees
    # Slope is the angle of steepest descent
    slope = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))

    # Calculate aspect in degrees (0-360, clockwise from north)
    # Aspect is the direction of steepest descent
    aspect = np.degrees(np.arctan2(-dx, dy))
    # Convert to 0-360 range (0 = North)
    aspect = np.where(aspect < 0, aspect + 360, aspect)

    return elevation, slope, aspect

# Generate terrain using simplex noise instead of random values
elevation, slope, aspect = generate_terrain_simplex(
    rows=grid_size[0],
    cols=grid_size[1],
    cell_size=1.0,
    min_elevation=100,
    max_elevation=1000,
    scale=100.0,
    seed=42
)

# Adjust humidity based on elevation (higher elevation = typically lower humidity)
elevation_normalized = (elevation - 100) / 900  # Normalize to 0-1 range

In [4]:
# Import necessary libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import random
import json
from scipy.stats import pearsonr
from deap import base, creator, tools, algorithms

# Import our model from the src directory
from src.model import CA

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

## Finding Available Fire Data

First, we'll identify the available fire datasets in the data directory that we can use for calibration.

In [5]:
fires = ['alabama', 'arizona']

## Setting Up the Calibration Framework

We'll define functions to evaluate model performance against real fire data using the Sørensen index.

In [6]:
def init_model_from_fire_data(fire, grid_size=(100, 100)):
    """Initialize a CA model from fire data"""
    # Create a new CA model
    model = CA(grid_size=grid_size)
    model.load_terrain_data(slope, aspect, elevation)
    model.load_mtbs_fire_data(fires[0])
    success = model.initialise_ndvi_from_data(fire)
    print(f"Model initialized from {fire} data: {success}")

    return model if success else None

def evaluate_model_performance(model, simulation_steps=20, params=None):
    """Run a model simulation and evaluate performance against actual fire data"""
    if params:
        # Set model parameters
        model.p0 = params.get('p0', 0.5)
        model.c1 = params.get('c1', 0.5)
        model.c2 = params.get('c2', 0.5)

    # Run the simulation
    history = model.run_simulation(simulation_steps)

    # Compare with actual burned area
    if model.actual_burned_area is not None:
        simulated_burned = (model.grid == 2).astype(int)  # Cells with state 2 are burnt

        # Calculate Sørensen index (Dice coefficient)
        true_positives = np.sum((simulated_burned == 1) & (model.actual_burned_area == 1))
        false_positives = np.sum((simulated_burned == 1) & (model.actual_burned_area == 0))
        false_negatives = np.sum((simulated_burned == 0) & (model.actual_burned_area == 1))

        sorensen = 2 * true_positives / (2 * true_positives + false_positives + false_negatives) if (2 * true_positives + false_positives + false_negatives) > 0 else 0

        return sorensen
    else:
        return 0.0  # No actual data to compare with

## Implementing the Genetic Algorithm for Parameter Optimization

Now we'll implement a genetic algorithm to find the optimal parameters for our fire spread model.

In [7]:
# Define parameter ranges
PARAM_RANGES = {
    'p0': (0.1, 0.9),   # Base ignition probability
    'c1': (0.1, 1.0),   # Wind effect parameter 1
    'c2': (0.1, 1.0)    # Wind effect parameter 2
}

# Define genetic algorithm fitness function
def evaluate_individual(individual, fire_folder, simulation_steps=20, grid_size=(100, 100)):
    """Evaluate fitness of a GA individual (parameter set)"""
    # Convert GA individual to parameter dictionary
    params = {
        'p0': individual[0],
        'c1': individual[1],
        'c2': individual[2]
    }

    # Initialize model
    model = init_model_from_fire_data(fire_folder, grid_size=grid_size)
    if not model:
        return (0.0,)  # Return tuple with single value for DEAP

    # Evaluate with these parameters
    sorensen = evaluate_model_performance(model, simulation_steps, params)

    return (sorensen,)  # Return tuple with single value for DEAP

# Setup genetic algorithm
def setup_genetic_algorithm():
    """Setup DEAP genetic algorithm framework"""
    # We want to maximize the Sørensen index
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()

    # Register attribute generators
    toolbox.register("attr_p0", random.uniform, PARAM_RANGES['p0'][0], PARAM_RANGES['p0'][1])
    toolbox.register("attr_c1", random.uniform, PARAM_RANGES['c1'][0], PARAM_RANGES['c1'][1])
    toolbox.register("attr_c2", random.uniform, PARAM_RANGES['c2'][0], PARAM_RANGES['c2'][1])

    # Register individual and population creation
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.attr_p0, toolbox.attr_c1, toolbox.attr_c2), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Register genetic operators
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    return toolbox

# Check for DEAP creator reset
if hasattr(creator, "FitnessMax"):
    del creator.FitnessMax
if hasattr(creator, "Individual"):
    del creator.Individual

## Running the Genetic Algorithm

Now we'll run the genetic algorithm to find the optimal parameters for our fire spread model.

In [ ]:
def run_genetic_algorithm(fire_folder, pop_size=30, n_gen=10, simulation_steps=20):
    """Run genetic algorithm to optimize model parameters"""
    print(f"Running genetic algorithm optimization for {fire_folder}...")

    # Setup GA
    toolbox = setup_genetic_algorithm()

    # Register evaluation function with the specific fire dataset
    toolbox.register("evaluate", evaluate_individual, fire_folder=fire_folder, simulation_steps=simulation_steps, grid_size=grid_size)

    # Initialize population
    pop = toolbox.population(n=pop_size)

    # Initialize statistics
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run GA
    hof = tools.HallOfFame(1)  # Keep track of best individual
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.2, ngen=n_gen,
                                      stats=stats, halloffame=hof, verbose=True)

    # Get best individual
    best_ind = hof[0]
    best_params = {
        'p0': best_ind[0],
        'c1': best_ind[1],
        'c2': best_ind[2]
    }
    best_fitness = best_ind.fitness.values[0]

    print(f"\nOptimization complete.")
    print(f"Best parameters: {best_params}")
    print(f"Best Sørensen index: {best_fitness:.4f}")

    return best_params, best_fitness, logbook

calibration_fire = fires[0]
best_params, best_fitness, logbook = run_genetic_algorithm(
    fire_folder=calibration_fire,
    pop_size=20,
    n_gen=5,     
    simulation_steps=30
)

In [10]:
best_params

{'p0': 0.8784926111834965, 'c1': 0.4406809394875182, 'c2': 0.5968365681459044}

## Advanced Model Calibration Methods

This section demonstrates more advanced methods for model calibration and parameter estimation using Bayesian methods, stochastic differential equations for environmental variables, and percolation theory integration.

### 1. Installing Required Dependencies

The Bayesian parameter estimation requires PyMC3, Arviz, and Theano packages.

In [ ]:
!pip install pymc arviz theano

### 2. Implementing Bayesian Parameter Estimation

Unlike the genetic algorithm which finds a single optimal set of parameters, Bayesian methods provide full posterior distributions that quantify parameter uncertainty.

In [8]:
def run_bayesian_parameter_estimation(fire_folder, n_samples=1000, tune=500, simulation_steps=20, grid_size=(100, 100)):
    """Run Bayesian parameter estimation for model calibration"""
    print(f"Running Bayesian parameter estimation for {fire_folder}...")
    
    # Initialize model
    model = init_model_from_fire_data(fire_folder, grid_size)
    if not model:
        return None, None
    
    # Define observations dictionary
    observations = {
        'simulation_steps': simulation_steps,
        'initial_fire_points': [(model.rows//2, model.cols//2)]  
    }
    
    # Set custom priors based on prior knowledge or GA results
    prior_params = {
        'p0_alpha': 3.0,    # Shape parameters for Beta distribution, centered around 0.6
        'p0_beta': 2.0,     # These parameters give a mean of 0.6
        'c1_mu': 0.5,       # Mean for Normal distribution
        'c1_sigma': 0.2,    # Standard deviation
        'c2_mu': 0.5,       # Mean for Normal distribution
        'c2_sigma': 0.2     # Standard deviation
    }
    
    # Run Bayesian parameter estimation
    print("Starting MCMC sampling - this may take some time...")
    trace, summary = model.bayesian_parameter_estimation(
        observations=observations,
        prior_params=prior_params,
        n_samples=n_samples,
        tune=tune
    )
    
    if trace is None:
        print("Error: Bayesian estimation failed. Please install required packages.")
        return None, None
    
    # Extract optimal parameters from posterior means
    best_params = {
        'p0': summary.loc['p0', 'mean'],
        'c1': summary.loc['c1', 'mean'],
        'c2': summary.loc['c2', 'mean']
    }
    
    print(f"\nBayesian estimation complete.")
    print(f"Optimal parameters (posterior means): {best_params}")
    print(f"Parameter 95% credible intervals:")
    print(f"p0: [{summary.loc['p0', 'hdi_3%']:.4f}, {summary.loc['p0', 'hdi_97%']:.4f}]")
    print(f"c1: [{summary.loc['c1', 'hdi_3%']:.4f}, {summary.loc['c1', 'hdi_97%']:.4f}]")
    print(f"c2: [{summary.loc['c2', 'hdi_3%']:.4f}, {summary.loc['c2', 'hdi_97%']:.4f}]")
    
    return best_params, trace

In [9]:
# Run Bayesian parameter estimation with a small number of samples for demonstration
calibration_fire = fires[0]
bayesian_params, trace = run_bayesian_parameter_estimation(
    fire_folder=calibration_fire,
    n_samples=500,    # Small sample size for demonstration
    tune=200,         # Small tuning phase for demonstration
    simulation_steps=20,
    grid_size=grid_size
)

Running Bayesian parameter estimation for alabama...
(187, 62)
Loaded burn perimeter shapefile: data/al3039808817220190514/al3039808817220190514_20190513_20190528_burn_bndy.shp
Loaded DNBR raster: data/al3039808817220190514/al3039808817220190514_20190513_20190528_dnbr.tif
Initializing from alabama fire
Loaded burn perimeter successfully
Resampling DNBR data from (299, 287) to (250, 250)
Estimated NDVI from DNBR data
Model initialized from alabama data: True
Starting MCMC sampling - this may take some time...
Error: This method requires PyMC3, Arviz, and Theano. Please install with:
pip install pymc3 arviz theano


TypeError: cannot unpack non-iterable NoneType object

### 3. Visualizing Parameter Posterior Distributions

One advantage of Bayesian estimation is that we get full posterior distributions, showing uncertainty in parameter estimates.

In [ ]:
import arviz as az

# Plot posterior distributions if trace is available
if trace is not None:
    # Plot trace and posterior distributions
    az.plot_trace(trace, var_names=['p0', 'c1', 'c2'])
    plt.tight_layout()
    plt.show()
    
    # Plot parameter correlations
    az.plot_pair(trace, var_names=['p0', 'c1', 'c2'])
    plt.tight_layout()
    plt.show()

NameError: name 'trace' is not defined

### 4. Comparing Calibration Methods

We can compare the parameters estimated by the genetic algorithm and the Bayesian approach.

In [ ]:
# Compare parameters from both methods
if bayesian_params and best_params:
    comparison_df = pd.DataFrame({
        'Genetic Algorithm': [best_params['p0'], best_params['c1'], best_params['c2']],
        'Bayesian Estimation (Mean)': [bayesian_params['p0'], bayesian_params['c1'], bayesian_params['c2']]
    }, index=['p0', 'c1', 'c2'])
    
    display(comparison_df)
    
    # Visualize parameter comparison
    comparison_df.plot(kind='bar', figsize=(10, 6))
    plt.title('Comparison of Parameter Estimation Methods')
    plt.ylabel('Parameter Value')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

### 5. Dynamic Environmental Conditions using Stochastic Differential Equations

We can make our simulations more realistic by allowing environmental conditions to evolve stochastically during the simulation.

In [ ]:
def run_sde_simulation(model, steps=30, dt=0.1):
    """Run simulation with stochastic environmental conditions"""
    history = [np.copy(model.grid)]
    env_history = []
    
    # Initial environmental conditions
    env_history.append({
        'wind_speed': model.wind_speed,
        'wind_direction': model.wind_direction,
        'temperature': model.temperature,
        'humidity': np.mean(model.humidity),
        'precipitation': model.precipitation
    })
    
    for step in range(steps):
        # Update environmental conditions stochastically
        env_conditions = model.update_environmental_conditions(dt=dt)
        env_history.append(env_conditions)
        
        # Update fire spread
        model.update()
        history.append(np.copy(model.grid))
        
        # Stop if no more burning cells
        if not np.any(model.grid == 1):
            print(f"Fire contained after {step+1} steps")
            break
    
    return history, env_history

def visualize_environmental_dynamics(env_history):
    """Visualize how environmental conditions change over the simulation"""
    env_df = pd.DataFrame(env_history)
    
    fig, axs = plt.subplots(3, 1, figsize=(12, 10), sharex=True)
    
    # Plot wind speed and direction
    axs[0].plot(env_df['wind_speed'], 'b-', label='Wind Speed (m/s)')
    axs[0].set_ylabel('Wind Speed (m/s)')
    axs[0].set_title('Wind Speed Evolution')
    axs[0].grid(True)
    
    axs[1].plot(env_df['wind_direction'], 'g-', label='Wind Direction (°)')
    axs[1].set_ylabel('Wind Direction (°)')
    axs[1].set_title('Wind Direction Evolution')
    axs[1].grid(True)
    
    # Plot temperature, humidity, and precipitation
    ax2 = axs[2]
    ax2.plot(env_df['temperature'], 'r-', label='Temperature (°F)')
    ax2.set_ylabel('Temperature (°F)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')
    ax2.set_title('Temperature, Humidity, and Precipitation Evolution')
    ax2.set_xlabel('Simulation Step')
    ax2.grid(True)
    
    ax3 = ax2.twinx()
    ax3.plot(env_df['humidity'], 'b--', label='Humidity (%)')
    ax3.plot(env_df['precipitation'], 'g-.', label='Precipitation (mm)')
    ax3.set_ylabel('Humidity (%) / Precipitation (mm)')
    
    # Add legend
    lines1, labels1 = ax2.get_legend_handles_labels()
    lines2, labels2 = ax3.get_legend_handles_labels()
    ax3.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Initialize model with optimal parameters (from either method)
optimal_params = bayesian_params if bayesian_params else best_params
model = init_model_from_fire_data(calibration_fire)

if model and optimal_params:
    # Set optimal parameters
    model.p0 = optimal_params['p0']
    model.c1 = optimal_params['c1']
    model.c2 = optimal_params['c2']
    
    # Set initial environmental conditions
    model.set_environmental_data(wind_speed=5.0, wind_direction=225.0, 
                               temperature=50, humidity=5, fire_direction=20)
    
    # Run simulation with stochastic environmental conditions
    print("Running simulation with stochastic environmental conditions...")
    history, env_history = run_sde_simulation(model, steps=30, dt=0.1)
    
    # Visualize fire spread
    model.visualize_simulation(history)
    
    # Visualize environmental dynamics
    visualize_environmental_dynamics(env_history)

### 6. Analyzing Percolation Effects on Fire Spread

The percolation threshold integration allows us to better model how fire spreads through heterogeneous landscapes.

In [ ]:
def visualize_percolation_thresholds(model):
    """Visualize percolation thresholds across the landscape"""
    # Calculate percolation thresholds for the entire grid
    pc_map = np.zeros((model.rows, model.cols))
    exceeds_map = np.zeros((model.rows, model.cols))
    
    for r in range(model.rows):
        for c in range(model.cols):
            pc, exceeds = model.calculate_percolation_threshold(r, c)
            pc_map[r, c] = pc
            exceeds_map[r, c] = 1 if exceeds else 0
    
    # Create visualization
    fig, axs = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot the NDVI (fuel density)
    im1 = axs[0].imshow(model.ndvi, cmap='YlGn')
    axs[0].set_title('Fuel Density (NDVI)')
    plt.colorbar(im1, ax=axs[0])
    
    # Plot the percolation thresholds
    im2 = axs[1].imshow(pc_map, cmap='plasma')
    axs[1].set_title('Percolation Thresholds')
    plt.colorbar(im2, ax=axs[1])
    
    # Plot areas that exceed threshold
    im3 = axs[2].imshow(exceeds_map, cmap='RdYlGn')
    axs[2].set_title('Areas Exceeding Percolation Threshold')
    plt.colorbar(im3, ax=axs[2], ticks=[0, 1], 
                 label='0: Below Threshold, 1: Above Threshold')
    
    plt.tight_layout()
    plt.show()
    
    return pc_map, exceeds_map

In [ ]:
# Analyze percolation thresholds in the model
if model:
    print("Analyzing percolation thresholds across the landscape...")
    pc_map, exceeds_map = visualize_percolation_thresholds(model)
    
    # Calculate statistics
    exceed_percentage = np.mean(exceeds_map) * 100
    avg_threshold = np.mean(pc_map)
    
    print(f"Average percolation threshold: {avg_threshold:.4f}")
    print(f"Percentage of landscape exceeding threshold: {exceed_percentage:.1f}%")
    
    # Check if there are natural firebreaks (areas below threshold surrounding higher-risk areas)
    from scipy import ndimage
    firebreak_kernel = np.ones((3, 3))
    potential_firebreaks = ndimage.binary_dilation(exceeds_map > 0) & ~(exceeds_map > 0)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(potential_firebreaks, cmap='Reds')
    plt.title('Potential Natural Firebreaks (Areas Below Threshold Adjacent to High-Risk Areas)')
    plt.colorbar(label='Potential Firebreak')
    plt.show()

### 7. Comparing Fire Spread Models: With and Without Percolation

We can compare how fire spreads with and without percolation theory to see the difference in patterns.

In [ ]:
def run_comparative_simulations(fire_folder, params, with_sde=False):
    """Run simulations with and without percolation for comparison"""
    # Initialize two identical models
    model1 = init_model_from_fire_data(fire_folder)  # With percolation (default)
    
    # Temporarily modify the calculate_ignition_probability method to disable percolation
    # by patching the method in a second model instance
    model2 = init_model_from_fire_data(fire_folder)  # Will disable percolation
    
    if not model1 or not model2:
        return None, None
    
    # Set parameters for both models
    for model in [model1, model2]:
        model.p0 = params['p0']
        model.c1 = params['c1']
        model.c2 = params['c2']
        model.set_environmental_data(wind_speed=5.0, wind_direction=225.0, 
                                  temperature=50, humidity=5, fire_direction=20)
    
    # Store original method reference
    original_method = model2.calculate_ignition_probability
    
    # Define a modified version without percolation effects
    def modified_ignition_probability(self, row, col):
        """Calculate ignition probability without percolation effects"""
        if not (0 <= row < self.rows and 0 <= col < self.cols):
            return 0.0
        
        if self.grid[row, col] != 0:  # Already burning or burnt
            return 0.0
        
        highest_veg_prob = 0.0
        has_burning_neighbors = False
        
        for dr in [-1, 0, 1]:
            for dc in [-1, 0, 1]:
                if dr == 0 and dc == 0:
                    continue
                nr, nc = row + dr, col + dc
                if 0 <= nr < self.rows and 0 <= nc < self.cols and self.grid[nr, nc] == 1:
                    highest_veg_prob = max(highest_veg_prob, 
                                          self.fuel_model[self.fuel_type[nr, nc], 
                                                         self.fuel_type[row, col]])
                    has_burning_neighbors = True
            if has_burning_neighbors:
                break
        
        if not has_burning_neighbors:
            return 0.0
        
        # Calculate standard environmental effects
        wind_effects = self.wind_effect(self.c1, self.c2)
        topography_effects = self.topography_effect(self.slope[row, col])
        humidity_effects = self.humidity_effect(humidity=self.humidity)
        temperature_effects = self.temperature_effect(temperature=self.temperature)
        precipitation_effect = self.precipitation_effect(self.precipitation)
        p_density = self.ndvi[row, col] * 0.5 + 0.5
        
        # Calculate base probability without percolation factor
        base_probability = self.p0 * (1+highest_veg_prob) * (1+p_density) * wind_effects * topography_effects
        
        # Apply temperature and moisture effects
        moisture_effect = 1.0 / (humidity_effects * precipitation_effect)
        moisture_effect = min(moisture_effect, 5.0)  # Cap the effect
        
        # Final probability without percolation
        adjusted_probability = base_probability * temperature_effects * moisture_effect
        
        return min(1, adjusted_probability)
    
    # Patch the method for model2 - this is a somewhat hacky way to replace the method
    import types
    model2.calculate_ignition_probability = types.MethodType(modified_ignition_probability, model2)
    
    # Run simulations
    print("Running simulation with percolation effects...")
    if with_sde:
        history1, _ = run_sde_simulation(model1)
    else:
        history1 = model1.run_simulation(30)
    
    print("Running simulation without percolation effects...")
    if with_sde:
        history2, _ = run_sde_simulation(model2)
    else:
        history2 = model2.run_simulation(30)
    
    # Restore original method to avoid side effects
    model2.calculate_ignition_probability = original_method
    
    return history1, history2

In [ ]:
# Run comparative simulations
if optimal_params:
    history_with_percolation, history_without_percolation = run_comparative_simulations(
        fire_folder=calibration_fire,
        params=optimal_params,
        with_sde=False  # Set to True to include stochastic environmental conditions
    )
    
    if history_with_percolation and history_without_percolation:
        # Visualize final fire perimeters side by side
        fig, axs = plt.subplots(1, 2, figsize=(16, 8))
        
        # With percolation
        final_with = history_with_percolation[-1]
        axs[0].imshow((final_with == 2), cmap='Reds')
        axs[0].set_title('Fire Spread WITH Percolation Effects')
        axs[0].axis('off')
        
        # Without percolation
        final_without = history_without_percolation[-1]
        axs[1].imshow((final_without == 2), cmap='Reds')
        axs[1].set_title('Fire Spread WITHOUT Percolation Effects')
        axs[1].axis('off')
        
        plt.tight_layout()
        plt.show()
        
        # Compare fire sizes
        burned_with = np.sum(final_with == 2)
        burned_without = np.sum(final_without == 2)
        print(f"Burned area WITH percolation: {burned_with} cells")
        print(f"Burned area WITHOUT percolation: {burned_without} cells")
        print(f"Difference: {abs(burned_with - burned_without)} cells 
({100*abs(burned_with - burned_without)/max(burned_with, burned_without):.1f}%)")

### 8. Saving and Loading Calibrated Model Parameters

Save the calibrated parameters for future use in other models and simulations.

In [ ]:
def save_model_parameters(params, method, file_path='calibrated_params.json'):
    """Save calibrated parameters to JSON file"""
    timestamp = pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
    data = {
        'parameters': params,
        'method': method,
        'fire_dataset': calibration_fire,
        'timestamp': timestamp
    }
    
    with open(file_path, 'w') as f:
        json.dump(data, f, indent=4)
    
    print(f"Parameters saved to {file_path}")

def load_model_parameters(file_path='calibrated_params.json'):
    """Load calibrated parameters from JSON file"""
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        print(f"Loaded parameters calibrated on {data['fire_dataset']} using {data['method']}")
        return data['parameters']
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None
    except json.JSONDecodeError:
        print(f"Error decoding JSON from {file_path}.")
        return None

In [ ]:
# Save parameters from both methods
if best_params:
    save_model_parameters(best_params, 'Genetic Algorithm', 'ga_params.json')
    
if bayesian_params:
    save_model_parameters(bayesian_params, 'Bayesian Estimation', 'bayes_params.json')